This notebook is for the preliminary trials of creating a basic classification model using tensor flow
the model will be a transfer learning model for a cnn network based on vgg 16 model found in the tensorflow keras models library 

In [8]:
!pip install -r required_packages.txt

  Using cached mlflow-2.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached types_PyYAML-6.0.12.20240917-py3-none-any.whl.metadata (1.6 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached dvc-3.56.0-py3-none-any.whl.metadata (18 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ensure-1.0.2-py2.py3-none-any.whl.metadata (10 kB)
  Using cached python_box-6.0.2-cp310-cp310-win_amd64.whl.metadata (7.7 kB)
  Using cached Flask_Cors-5.0.0-py2.py3-none-any.whl.metadata (5.5 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached databricks_cli-0.18.0-py2.py3-none-any.whl.metadata (4.0 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl.metadata 

In [5]:
import os
%pwd
os.chdir('../')
%pwd

'c:\\Users\\Omar Hady\\Data Projects\\Machine Learning\\End-To-End Cancer Evaluation\\End-To-End-Cancer-Classification-Using-MLflow'

In [9]:
#Create data class for base model (update entity)
from dataclasses import dataclass
from pathlib import Path

@dataclass
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int


In [16]:
#Create a method in the configuration management file (uodate configuration manager)

from src.Chest_Cancer_Classification.constants import *
from src.Chest_Cancer_Classification.utils.common import create_directories, read_yaml
from src.Chest_Cancer_Classification.entity.config_entity import DataIngestionConfiguration


class ConfigurationManager:
    '''
    General Configuration class containing configuration methods for every component
    '''
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    
    def get_base_model(self)-> BaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weight= self.params.WEIGHTS,
            params_classes= self.params.CLASSES)
        return base_model_config

    







In [17]:
import os
import zipfile
import tensorflow as tf
from urllib import request

In [18]:
#Create the component 

class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    def create_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weight,
            include_top = self.config.params_include_top

        )

        self.save_model(path= self.config.base_model_path, model=self.model)

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        return model.save(path)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        ''' 
        A static method to create the full model using a base model from transfer learning

        Arguments:
            model (keras.Model object): base model
            classes: no. of classes that the model will classify
            freeze_all (bool): if True freezes all the layers of the base model without updating
            freeze_till (int) : if given, the number of layers that shouldn't be updated 
                                and the layers after it will be updated
            learning_rate (float): model learning rate for updating
        '''

        if freeze_all:
            for layers in model.layers:
                model.trainable = False
        elif (freeze_till is not None):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        predict_in = tf.keras.layers.Dense(units=classes, activation = "softmax")(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=predict_in )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)



In [19]:
#pipeline

config = ConfigurationManager()
prepare_model_config = config.get_base_model()
base_model = BaseModel(config=prepare_model_config)
base_model.create_base_model()
base_model.update_base_model()



[2024-11-19 02:06:00,492 - INFO - common - config\config.yaml loaded successfully]
[2024-11-19 02:06:00,495 - INFO - common - params.yaml loaded successfully]
[2024-11-19 02:06:00,495 - INFO - common - created directory at: artifacts]
[2024-11-19 02:06:00,496 - INFO - common - created directory at: artifacts/prepare_base_model]
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
[2024-11-19 02:06:08,255 - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

[2024-11-19 02:06:08,372 - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
